In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime

#### 定义常量

In [2]:
rnn_unit = 10 #隐藏层
input_size = 7
output_size = 1
lr = 0.0006 #学习率

#### 导入选择数据

In [3]:
f = open('dataset.csv')
df = pd.read_csv(f)
data = df.iloc[:,2:10].values #取第3-10列数据
#df.iloc[1:3,7]

In [4]:
len(data)

6109

#### 从导入数据内选取0-5800行作为创建训练集

In [5]:
def get_train_data(batch_size = 60 , time_step = 20 , train_begin = 0 , train_end = 5800):
    batch_index = []
    data_train = data[train_begin:train_end] #选取训练数据范围0至5800
    normalized_train_data = (data_train - np.mean(data_train,axis=0)) / np.std(data_train,axis=0)#标准化数据
    print(len(normalized_train_data))
    train_x,train_y=[],[]
    for i in range(len(normalized_train_data)-time_step):  
        #for i in range 循环的数列为这个范围 range(0, 5780)
        if i % batch_size==0: #如果（0循环至8780中除以20为0）
            batch_index.append(i) #批数据索引追加这个导致为0的数字
        x = normalized_train_data[i:i+time_step,:7] #i至i+20行的1至7列转换成列
        y = normalized_train_data[i:i+time_step,7,np.newaxis] #改变维度下面有例子
        train_x.append(x.tolist()) #train_x追加x的转换成列表形式的内容
        train_y.append(y.tolist()) #train_y追加y的转换成列表形式的内容
    batch_index.append((len(normalized_train_data)-time_step)) #barch_index内追加5800
    return batch_index,train_x,train_y

Aarry 操作 http://blog.csdn.net/sunny2038/article/details/9023797

#### ——————————获取测试集——————————

In [6]:
def get_test_data(time_step = 20 ,test_begin=5800):  #总共有6109行
    data_test = data[test_begin:] #309行
    mean = np.mean(data_test,axis = 0) # data_test 将测试数据以列方式计算平均数
    std = np.std(data_test,axis = 0) # std = 测试数据列的标准差
    normalized_test_data = (data_test-mean)/std #标准测试数据 = （测试数据-平均值）/标准差
    size = (len(normalized_test_data)+time_step)//time_step #309+20-1//20
    test_x,test_y = [],[]
    for i in range(size-1):
        x = normalized_test_data[i+time_step:(i+1)*time_step,:7]
        y = normalized_test_data[i*time_step:(i+1)*time_step,7]
        test_x.append(x.tolist())
        test_y.extend(y)       
    test_x.append((normalized_test_data[(i+1)*time_step:,:7]).tolist()) #追加
    test_y.extend((normalized_test_data[(i+1)*time_step:,7]).tolist()) #合并
    return mean,std,test_x,test_y

#### ——————————————————定义神经网络变量——————————————————

In [7]:

#输入层、输出层权重、偏置

weights={
         'in':tf.Variable(tf.random_normal([input_size,rnn_unit])),
         'out':tf.Variable(tf.random_normal([rnn_unit,1]))
        }
biases={
        'in':tf.Variable(tf.constant(0.1,shape=[rnn_unit,])),
        'out':tf.Variable(tf.constant(0.1,shape=[1,]))
       }

#### 构建神经网络

![CSDN图标][csdn]
[csdn]: http://static.open-open.com/lib/uploadImg/20161020/20161020103810_609.png "这是CSDN的图标"
#### 图一

In [8]:
def lstm(X):
    batch_size = tf.shape(X)[0]
    time_step = tf.shape(X)[1]
    w_in = weights['in'] #权重
    b_in = biases['in'] #偏置
    #需要将tensor转成2维进行计算，计算后的结果作为隐藏层的输入
    #原始的 X 是 3 维数据, 我们需要把它变成 2 维数据才能使用 weights 的矩阵乘法
    # # X ==> (128 batches * 28 steps, 28 inputs)
    #需要将tensor转成2维进行计算，计算后的结果作为隐藏层的输入
    input = tf.reshape(X,[-1,input_size])   #tf.reshape(x,[-1,7])
    input_rnn = tf.matmul(input,w_in)+b_in # X_in = W*X + b
    # X_in ==> (128 batches, 28 steps, 128 hidden) 换回3维
    input_rnn = tf.reshape(input_rnn,[-1,time_step,rnn_unit]) #将tensor转成3维，作为lstm cell的输入
    cell = tf.contrib.rnn.BasicLSTMCell(rnn_unit)   #num_units:图一中ht的维数，如果num_units=10,那么ht就是10维行向量
    init_state = cell.zero_state(batch_size,dtype = tf.float32)
    output_rnn,final_states = tf.nn.dynamic_rnn(cell, input_rnn,initial_state=init_state, dtype=tf.float32)  #output_rnn是记录lstm每个输出节点的结果，final_states是最后一个cell的结果     

    output=tf.reshape(output_rnn,[-1,rnn_unit]) #作为输出层的输入
    w_out=weights['out']
    b_out=biases['out']
    pred=tf.matmul(output,w_out)+b_out
    return pred,final_states

#### 训练模型

In [9]:
def train_lstm(batch_size=80,time_step=15,train_begin=2000,train_end=5800):
    X=tf.placeholder(tf.float32, shape=[None,time_step,input_size])
    Y=tf.placeholder(tf.float32, shape=[None,time_step,output_size])
    batch_index,train_x,train_y=get_train_data(batch_size,time_step,train_begin,train_end)
    pred,_=lstm(X)
    #损失函数
    loss=tf.reduce_mean(tf.square(tf.reshape(pred,[-1])-tf.reshape(Y, [-1])))
    train_op=tf.train.AdamOptimizer(lr).minimize(loss)
    saver=tf.train.Saver(tf.global_variables(),max_to_keep=15)
    starttime = datetime.datetime.now()
    #module_file = tf.train.latest_checkpoint()    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        #saver.restore(sess, module_file)
        #重复训练10000次
        for i in range(2000):
            for step in range(len(batch_index)-1):
                _,loss_=sess.run([train_op,loss],feed_dict={X:train_x[batch_index[step]:batch_index[step+1]],Y:train_y[batch_index[step]:batch_index[step+1]]})
            print(i,loss_)
            if i % 200==0:
                print("保存模型：",saver.save(sess,'stock2.model',global_step=i))

                

                endtime = datetime.datetime.now()
                print ((endtime - starttime).seconds)
                

In [10]:
train_lstm()

3800
0 0.997316
保存模型： stock2.model-0
0
1 0.25518
2 0.0807025
3 0.0441647
4 0.0457441
5 0.0530179
6 0.0568015
7 0.0570582
8 0.0548586
9 0.0510863
10 0.0464994
11 0.0416733
12 0.0369803
13 0.0326275
14 0.0287086
15 0.0252459
16 0.0222206
17 0.0195918
18 0.0173126
19 0.015337
20 0.0136254
21 0.0121459
22 0.0108751
23 0.00979558
24 0.0088902
25 0.00812917
26 0.00744403
27 0.00677609
28 0.00615837
29 0.0056073
30 0.00512979
31 0.00470387
32 0.00434594
33 0.00400576
34 0.00375705
35 0.00343284
36 0.00337237
37 0.00278819
38 0.00338928
39 0.0018566
40 0.00248692
41 0.00246447
42 0.00250183
43 0.00173512
44 0.00237518
45 0.00153158
46 0.00174147
47 0.0021079
48 0.00176304
49 0.00141031
50 0.00139433
51 0.00282972
52 0.00155584
53 0.00184832
54 0.0016518
55 0.00327666
56 0.00162993
57 0.00263336
58 0.00197388
59 0.00326741
60 0.001544
61 0.00237168
62 0.00157227
63 0.00219554
64 0.00129189
65 0.00161436
66 0.00126107
67 0.0014978
68 0.00116947
69 0.00121338
70 0.00115603
71 0.00118507
72 0.0011

540 0.000841701
541 0.000406078
542 0.00129924
543 0.00178937
544 0.000531428
545 0.000650738
546 0.000652472
547 0.000440576
548 0.001728
549 0.00125234
550 0.000419188
551 0.00101075
552 0.000471839
553 0.00117087
554 0.0018286
555 0.000578385
556 0.000769844
557 0.000999072
558 0.000706816
559 0.00211083
560 0.00125393
561 0.000369219
562 0.00104549
563 0.000509751
564 0.00139712
565 0.00221459
566 0.000554674
567 0.000803837
568 0.000829643
569 0.000401743
570 0.00203897
571 0.00145151
572 0.000426692
573 0.00134942
574 0.000485018
575 0.00121348
576 0.00204286
577 0.000588993
578 0.00085901
579 0.00119569
580 0.000659493
581 0.00207675
582 0.0013008
583 0.000352851
584 0.00105265
585 0.000555524
586 0.00123461
587 0.00205917
588 0.000582323
589 0.000670418
590 0.0008043
591 0.000351707
592 0.00173919
593 0.00137574
594 0.000391752
595 0.001087
596 0.000480048
597 0.000981558
598 0.00178216
599 0.000583372
600 0.000746287
保存模型： stock2.model-600
220
601 0.00100689
602 0.000621907
60

1061 0.00107519
1062 0.000415924
1063 0.00113932
1064 0.000390338
1065 0.000760324
1066 0.00102676
1067 0.000256649
1068 0.00119329
1069 0.000495793
1070 0.000807626
1071 0.00120003
1072 0.000353671
1073 0.00106445
1074 0.0004956
1075 0.000602014
1076 0.00114376
1077 0.000253459
1078 0.00112816
1079 0.0006205
1080 0.000656583
1081 0.00127577
1082 0.000340877
1083 0.000956054
1084 0.000618521
1085 0.000466265
1086 0.00122382
1087 0.000301323
1088 0.0010278
1089 0.000744607
1090 0.000519829
1091 0.00129952
1092 0.000374672
1093 0.000826482
1094 0.000744659
1095 0.000362003
1096 0.00126269
1097 0.000391887
1098 0.000905038
1099 0.000855618
1100 0.000408763
1101 0.00127481
1102 0.000447946
1103 0.000689034
1104 0.00086122
1105 0.000294338
1106 0.00126036
1107 0.000513498
1108 0.000772959
1109 0.000944185
1110 0.000330598
1111 0.00120892
1112 0.000550979
1113 0.000555667
1114 0.000958216
1115 0.000264344
1116 0.00121958
1117 0.000653331
1118 0.000643107
1119 0.00100432
1120 0.000288599
1121

1548 0.000255468
1549 0.000541851
1550 0.000221529
1551 0.000621702
1552 0.000878561
1553 0.000199939
1554 0.000468651
1555 0.000235175
1556 0.000865523
1557 0.000740077
1558 0.000251678
1559 0.000531518
1560 0.000220455
1561 0.000593813
1562 0.00101188
1563 0.00019344
1564 0.000514307
1565 0.00026745
1566 0.000865371
1567 0.000917296
1568 0.000262545
1569 0.000602172
1570 0.000253499
1571 0.000580176
1572 0.00123395
1573 0.00021521
1574 0.00056031
1575 0.000392456
1576 0.000650472
1577 0.00103152
1578 0.000394768
1579 0.000478645
1580 0.000366564
1581 0.000299236
1582 0.00121081
1583 0.000364572
1584 0.000350141
1585 0.000479119
1586 0.000348594
1587 0.000843197
1588 0.000581648
1589 0.000299471
1590 0.000398306
1591 0.000214814
1592 0.00109797
1593 0.000496536
1594 0.000270659
1595 0.000493275
1596 0.000262779
1597 0.000769891
1598 0.000747192
1599 0.000230732
1600 0.000438147
保存模型： stock2.model-1600
580
1601 0.000202234
1602 0.0010666
1603 0.00066605
1604 0.000247079
1605 0.00055553